# Hồi quy tuyến tính đa biến
Trong bài lab này, chúng ta sẽ dự đoán lợi nhuận của startup dự theo các thông số sau: 

- Số tiền bỏ ra cho các hoạt động R&D
- Chi phí quản lý hành chính
- Số tiền "đốt" cho các hoạt động marketing
- Vị trí địa lý của công ty (thuộc tiểu bang nào)

## Import các thư viện

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Import tập dữ liệu

In [2]:
dataset = pd.read_csv('data/50_Startups.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

## Mã hóa dữ liệu phân loại

Sử dụng **One-hot Encoding** và **ColumnTransformer()** để mã hóa các biến độc lập. 

**ColumnTransformer()** cho phép các cột hoặc các tập hợp con cột khác nhau của input được chuyển đổi riêng biệt và các feature do mỗi transformer tạo ra sẽ được nối với nhau để tạo thành một không gian feature duy nhất. Điều này rất hữu ích cho dữ liệu không đồng nhất hoặc dữ liệu dạng cột, kết hợp một số cơ chế trích xuất đặc trưng hoặc biến đổi thành một transformer duy nhất.

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
X

array([[0.0, 0.0, 1.0, 165349.2, 136897.8, 471784.1],
       [1.0, 0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [0.0, 1.0, 0.0, 153441.51, 101145.55, 407934.54],
       [0.0, 0.0, 1.0, 144372.41, 118671.85, 383199.62],
       [0.0, 1.0, 0.0, 142107.34, 91391.77, 366168.42],
       [0.0, 0.0, 1.0, 131876.9, 99814.71, 362861.36],
       [1.0, 0.0, 0.0, 134615.46, 147198.87, 127716.82],
       [0.0, 1.0, 0.0, 130298.13, 145530.06, 323876.68],
       [0.0, 0.0, 1.0, 120542.52, 148718.95, 311613.29],
       [1.0, 0.0, 0.0, 123334.88, 108679.17, 304981.62],
       [0.0, 1.0, 0.0, 101913.08, 110594.11, 229160.95],
       [1.0, 0.0, 0.0, 100671.96, 91790.61, 249744.55],
       [0.0, 1.0, 0.0, 93863.75, 127320.38, 249839.44],
       [1.0, 0.0, 0.0, 91992.39, 135495.07, 252664.93],
       [0.0, 1.0, 0.0, 119943.24, 156547.42, 256512.92],
       [0.0, 0.0, 1.0, 114523.61, 122616.84, 261776.23],
       [1.0, 0.0, 0.0, 78013.11, 121597.55, 264346.06],
       [0.0, 0.0, 1.0, 94657.16, 145077.58

## Chia tập dữ liệu thành Training set và Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Huấn luyện mô hình Hồi quy tuyến tính đa biến trên Training set

In [6]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

## Dự đoán kết quả Test set

In [7]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2) # set print precision to 2 decimal places
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[126362.88 134307.35]
 [ 84608.45  81005.76]
 [ 99677.49  99937.59]
 [ 46357.46  64926.08]
 [128750.48 125370.37]
 [ 50912.42  35673.41]
 [109741.35 105733.54]
 [100643.24 107404.34]
 [ 97599.28  97427.84]
 [113097.43 122776.86]]


## Đưa ra một dự đoán duy nhất (chẳng hạn: lợi nhuận của một startup với R&D Spend = 160000, Administration Spend = 130000, Marketing Spend = 300000 and State = 'California')

In [8]:
print(regressor.predict([[1, 0, 0, 160000, 130000, 300000]]))

[182943.06]


**Lưu ý quan trọng 1:** Lưu ý rằng tất cả các giá trị của các feature đều được nhập trong một cặp dấu ngoặc vuông kép. Đó là bởi vì phương thức "predict" luôn cần một mảng 2D là định dạng của các input. Và việc đặt các giá trị vào một cặp dấu ngoặc vuông kép làm cho input chính xác là một mảng 2D. Chỉ cần đặt:

$1, 0, 0, 160000, 130000, 300000 \rightarrow \textrm{scalars}$

$[1, 0, 0, 160000, 130000, 300000] \rightarrow \textrm{1D array}$

$[[1, 0, 0, 160000, 130000, 300000]] \rightarrow \textrm{2D array}$

**Lưu ý quan trọng 2:** Cũng lưu ý rằng trạng thái "California" không được nhập dưới dạng string trong cột cuối cùng mà là "1, 0, 0" trong ba cột đầu tiên. Đó là bởi vì phương thức dự đoán yêu cầu các giá trị one-hot coded của trạng thái và như chúng ta thấy ở hàng thứ hai của ma trận các feature X, "California" được mã hóa thành "1, 0, 0". Và hãy chú ý bao hàm các giá trị này trong ba cột đầu tiên chứ không phải ba cột cuối cùng, vì các biến giả luôn được tạo trong các cột đầu tiên.

## Nhận phương trình hồi quy tuyến tính cuối cùng với giá trị của các hệ số

In [9]:
print(regressor.coef_)
print(regressor.intercept_)

[-3.15e+02  6.24e+02 -3.08e+02  8.06e-01 -6.88e-02  2.99e-02]
54343.29984944819



**Lưu ý quan trọng:** Để có được các hệ số này, chúng ta gọi thuộc tính "coef_" và "intercept_" từ đối tượng hồi quy. Các thuộc tính trong Python khác với các phương thức và thường trả về một giá trị đơn giản hoặc một mảng các giá trị.